In [ ]:
# bdunaganGPT
# See bdunagan.com for more details.
# Based on Andrej Karpathy's YouTube video building GPT from scratch: https://karpathy.ai/zero-to-hero.html.
# MIT license (Karpathy's is as well).
import math
import time
import torch
import torch.nn as nn
from torch.nn import functional as F
import matplotlib.pyplot as plt
%matplotlib inline

# Set the seed for reproducibility
torch.manual_seed(1337)

# Set up loss tracking
losses = []
eval_iters = 100
eval_interval = 100
max_new_tokens = 100
loss_steps = []
loss_values = []

# Set up for toggling features to experiment with loss
enable_residual_connections = True
enable_learned_positional_encoding = True
enable_pytorch_multihead_attention = True

# Set up the device for running the model
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Import data set
!wget https://raw.githubusercontent.com/bdunagan/bdunaganGPT/main/bdunagan.com.txt
with open('bdunagan.com.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"vocab_size {vocab_size}")
print(f"tokens {len(text)}")
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head, block_size, dropout, device):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        self.block_size = block_size
        self.device = device
        if enable_pytorch_multihead_attention:
          # Pytorch Multihead Attention version (https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html):
          self.sa = nn.MultiheadAttention(n_embd, n_head, bias=False, device=device, batch_first=True)
        else:
          # Karpathy's original MultiheadAttention version:
          self.sa = MultiHeadAttention(n_embd, n_head, block_size, dropout)
        self.ffwd = FeedForward(n_embd, dropout)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        if enable_pytorch_multihead_attention:
          # Pytorch version:
          B, T, C = x.shape
          y = self.ln1(x)
          attn_mask = nn.Transformer.generate_square_subsequent_mask(T).to(self.device)
          output, _ = self.sa(y, y, y, attn_mask=attn_mask, need_weights=False, is_causal=True)
          if enable_residual_connections:
            x = x + output
            x = x + self.ffwd(self.ln2(x))
          else:
            x = output
            x = self.ffwd(self.ln2(x))
        else:
          # Karpathy's original version:
          if enable_residual_connections:
            x = x + self.sa(self.ln1(x))
            x = x + self.ffwd(self.ln2(x))
          else:
            x = self.sa(self.ln1(x))
            x = self.ffwd(self.ln2(x))
        return x

# Positional encoding using sin/cos approach:
# https://pytorch.org/tutorials/beginner/transformer_tutorial.html
class PositionalEncoding(nn.Module):

    def __init__(self, n_embd, dropout, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, n_embd, 2) * (-math.log(10000.0) / n_embd))
        pe = torch.zeros(max_len, 1, n_embd)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

# Karpathy's Head version:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, n_embd, n_head, block_size, dropout):
        super().__init__()
        head_size = n_embd // n_head
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

# Karpathy's MultiheadAttention version:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, n_embd, n_head, block_size, dropout):
        super().__init__()
        self.heads = nn.ModuleList([Head(n_embd, n_head, block_size, dropout) for _ in range(n_head)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, n_embd, n_head, n_layer, block_size, dropout, device):
        super().__init__()
        self.device = device
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)

        if enable_learned_positional_encoding:
          # Karpathy's original learned weights version of positional encoding:
          self.position_embedding_table = nn.Embedding(block_size, n_embd)
        else:
          # sin/cos version of positional encoding:
          self.position_embedding_table = PositionalEncoding(n_embd, dropout)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head, block_size, dropout, device) for _ in range(n_layer)])

        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        x = tok_emb
        if enable_learned_positional_encoding:
          # Karpathy's original learned positional encoding:
          pos_emb = self.position_embedding_table(torch.arange(T, device=self.device)) # (T,C)
          x = tok_emb + pos_emb # (B,T,C)
        else:
          # Positional encoding using sin/cos:
          pos_emb = self.position_embedding_table(tok_emb) # (T,C)
          x = pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, block_size, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

class GPTTest:
    def __init__(self, batch_size, block_size, max_iters, learning_rate, n_embd, n_head, n_layer, dropout, device):
        super().__init__()
        self.block_size = block_size
        self.batch_size = batch_size
        self.max_iters = max_iters
        self.learning_rate = learning_rate
        self.n_embd = n_embd
        self.n_head = n_head
        self.n_layer = n_layer
        self.dropout = dropout
        self.device = device
        self.model = BigramLanguageModel(n_embd=self.n_embd, n_head=self.n_head, n_layer=self.n_layer, block_size=self.block_size, dropout=self.dropout, device=self.device)
        self.m = self.model.to(self.device)
        print(f"parameters {sum(p.numel() for p in self.model.parameters())}")
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.learning_rate)

    # data loading
    def get_batch(self, split):
        # generate a small batch of data of inputs x and targets y
        data = train_data if split == 'train' else val_data
        ix = torch.randint(len(data) - self.block_size, (self.batch_size,))
        x = torch.stack([data[i:i+self.block_size] for i in ix])
        y = torch.stack([data[i+1:i+self.block_size+1] for i in ix])
        x, y = x.to(self.device), y.to(self.device)
        return x, y

    @torch.no_grad()
    def estimate_loss(self):
        out = {}
        self.model.eval()
        for split in ['train', 'val']:
            losses = torch.zeros(eval_iters)
            for k in range(eval_iters):
                X, Y = self.get_batch(split)
                logits, loss = self.model(X, Y)
                losses[k] = loss.item()
            out[split] = losses.mean()
        self.model.train()
        return out

    def run(self):
        start_time = time.time()
        for iter in range(self.max_iters):
            # every once in a while evaluate the loss on train and val sets
            if iter % eval_interval == 0:
              debug_loss = self.estimate_loss()
              print(f"[{(time.time() - start_time):.0f}s] step {iter}: train loss {debug_loss['train']:.4f}, val loss {debug_loss['val']:.4f}")

            # sample a batch of data
            xb, yb = self.get_batch('train')

            # evaluate the loss
            logits, loss = self.model(xb, yb)
            self.optimizer.zero_grad(set_to_none=True)
            loss.backward()
            self.optimizer.step()

            loss_steps.append(iter)
            loss_values.append(loss.log10().item())

        # Evaluate the final loss
        current_losses = self.estimate_loss()
        # Append to loss tracking for evaluating hyperparameters.
        losses.append({
            "parameters": f"{sum(p.numel() for p in self.m.parameters())}",
            "batch_size": self.batch_size,
            "block_size": self.block_size,
            "max_iters": self.max_iters,
            "learning_rate": self.learning_rate,
            "n_embd": self.n_embd,
            "n_head": self.n_head,
            "n_layer": self.n_layer,
            "dropout": self.dropout,
            "time": f"{(time.time() - start_time):.0f}",
            "train_loss": f"{current_losses['train']:.3f}",
            "val_loss": f"{current_losses['val']:.3f}",
        })
        print(f"[{(time.time() - start_time):.0f}s]: {losses[-1]}")

        # Generate from the model
        context = torch.zeros((1, 1), dtype=torch.long, device=self.device)
        print(decode(self.m.generate(context, block_size=self.block_size, max_new_tokens=max_new_tokens)[0].tolist()))

# GPU Test from blog: uncomment the following line and run the different available hardware accelerators:
# test = GPTTest(batch_size=64, block_size=128, max_iters=1000, learning_rate=1e-3, n_embd=64, n_head=8, n_layer=16, dropout=0.0, device=device)
# test.run()

# Small set of combinations with parameters, training time on a T4, and validation loss noted in comments:
# test = GPTTest(batch_size=8, block_size=32, max_iters=1000, learning_rate=1e-2, n_embd=8, n_head=4, n_layer=4, dropout=0.0, device=device) # 5k - 19s - 2.627
# test = GPTTest(batch_size=16, block_size=128, max_iters=1000, learning_rate=1e-3, n_embd=16, n_head=8, n_layer=8, dropout=0.1, device=device) # 31k - 36s - 2.588
# test = GPTTest(batch_size=32, block_size=128, max_iters=1000, learning_rate=1e-4, n_embd=32, n_head=16, n_layer=16, dropout=0.2, device=device) # 211k - 77s - 2.770
# test = GPTTest(batch_size=64, block_size=256, max_iters=1000, learning_rate=1e-5, n_embd=64, n_head=32, n_layer=32, dropout=0.3, device=device) # 1.6m - 1218s - 3.253
# test.run()

# Model for changing transformer toggles:
# test = GPTTest(batch_size=16, block_size=32, max_iters=2000, learning_rate=1e-3, n_embd=64, n_head=4, n_layer=4, dropout=0.0, device=device)
# test.run()

# Example of overfitting, where training loss and validation loss diverge:
# test = GPTTest(batch_size=64, block_size=128, max_iters=3000, learning_rate=1e-3, n_embd=64, n_head=8, n_layer=16, dropout=0.0, device=device)
# test.run()

# Train and run inference on a 10-million parameter model:
test = GPTTest(batch_size=32, block_size=128, max_iters=1400, learning_rate=1e-3, n_embd=128, n_head=4, n_layer=52, dropout=0.1, device=device)
test.run()

In [ ]:
# Plot loss
plt.plot(loss_steps, loss_values)